In [1]:
import pandas as pd
import altair as alt
import numpy as np
alt.data_transformers.enable("json")
import matplotlib.pyplot as plt

import scipy

# User inputs

In [2]:
start_date = '20221130'
end_date = '20230517'

tidy_dataset_fn = f"../sos/tidy_df_30Min_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_dataset_5min_fn = f"../sos/tidy_df_{start_date}_{end_date}_noplanar_fit.parquet"
tidy_daily_dataset_output_fn = f"tidy_df_daily_{start_date}_{end_date}_noplanar_fit.parquet"

# Load data

In [3]:
try:
    tidy_df_30Min = pd.read_parquet(
        tidy_dataset_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_30Min['time'] = pd.to_datetime(tidy_df_30Min['time'])

try:
    tidy_df_5Min = pd.read_parquet(
        tidy_dataset_5min_fn
    )
except FileNotFoundError:
    print("No file such file exists for these dates.")
tidy_df_5Min['time'] = pd.to_datetime(tidy_df_5Min['time'])

# Examine pot. virtual temperature gradient

In [4]:
np.abs(tidy_df_30Min[tidy_df_30Min.measurement == 'temperature gradient'].value).min()

2.611622715868907e-07

In [5]:
alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_line().encode(
    x = 'time:T',
    y = 'value:Q',
    column='height:O'
) & alt.Chart(
    tidy_df_30Min.query("measurement == 'temperature gradient'").query("height < 5")
).mark_bar().encode(
    alt.X('value:Q').bin(step=0.1),
    alt.Y("count():Q"),    
    alt.Column('height:O')
)

alt.VConcatChart(...)

In [6]:
src = tidy_df_30Min.query("variable == 'temp_gradient_3m_c'")
neutral_times = src[src['value'].abs() < 0.01].time

In [7]:
alt.Chart(
    tidy_df_30Min[
        tidy_df_30Min.time.isin(neutral_times.sample(16))
    ].query("measurement == 'wind speed'").query("tower == 'c'")
).mark_line().encode(
    alt.X("value:Q").title("Wind speed (m/s)").sort('-y'),
    alt.Y("height:Q").title("Height (m)"),
    alt.Facet("time:O", columns=8)
).properties(width = 125, height = 125)

alt.Chart(...)

In [8]:
src = tidy_df_30Min[
    tidy_df_30Min.time.isin(neutral_times)
].query("tower == 'c'")
src = src[src.measurement.isin([
    'wind speed',
    'shear velocity'
])]
src = src[~src.variable.str.contains("predicted")]
src = src.pivot_table(index=['time'], values='value', columns='variable')
src.head()

variable,spd_10m_c,spd_15m_c,spd_20m_c,spd_2m_c,spd_3m_c,spd_5m_c,u*_10m_c,u*_15m_c,u*_20m_c,u*_2m_c,u*_3m_c,u*_5m_c
time,,,,,,,,,,,,
2022-12-02 10:30:00,5.852733,5.990304,6.163234,5.169695,5.503640,5.802035,0.771105,0.711912,0.749025,0.772614,0.748710,0.600726
2022-12-02 11:00:00,5.108618,5.132134,5.137251,4.701602,4.848611,5.051483,0.792359,0.733981,0.720212,0.809777,0.727379,0.728357
2022-12-02 11:30:00,4.945608,5.086113,5.158172,4.254836,4.424224,4.764127,0.827612,0.762404,0.772660,0.802487,0.794172,0.635123
2022-12-02 12:00:00,6.954772,7.209726,7.195507,5.955409,6.454049,6.821709,0.922849,1.007616,1.199078,0.840452,0.820263,0.595426
2022-12-02 12:30:00,4.535348,4.960222,4.781742,4.116060,4.270485,4.554066,0.813880,0.840683,0.887759,0.753841,0.708530,0.631443


# Solve for $z_0$ assuming $d = 0$

https://www.eol.ucar.edu/content/calculation-roughness-length-and-displacement-height

In [9]:
von_karman = 0.4

In [10]:
d = 0.1
src['z0_2m_c'] = (1 - d)/np.exp(src['spd_2m_c']*von_karman/src['u*_2m_c'])
src['z0_3m_c'] = (2 - d)/np.exp(src['spd_3m_c']*von_karman/src['u*_3m_c'])
src['z0_5m_c'] = (4 - d)/np.exp(src['spd_5m_c']*von_karman/src['u*_5m_c'])
src['z0_10m_c'] = (9 - d)/np.exp(src['spd_10m_c']*von_karman/src['u*_10m_c'])
src['z0_15m_c'] = (14 - d)/np.exp(src['spd_15m_c']*von_karman/src['u*_15m_c'])
src['z0_20m_c'] = (19 - d)/np.exp(src['spd_20m_c']*von_karman/src['u*_20m_c'])

In [11]:
alt.Chart(src).transform_fold(
    ['z0_2m_c', 'z0_3m_c', 'z0_5m_c', 'z0_10m_c', 'z0_15m_c', 'z0_20m_c']
).mark_boxplot().encode(
    alt.X("value:Q").scale(type='log'),
    alt.Row('key:O')
).properties(height = 100)

alt.Chart(...)

In [64]:
src_weekly_averages_mean_and_median = src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1W')).agg(['mean', 'median']).copy()
src_weekly_averages_mean_and_median.columns = src_weekly_averages_mean_and_median.columns.to_flat_index()
src_weekly_averages_mean_and_median.columns = pd.Series(src_weekly_averages_mean_and_median.columns).apply(lambda tup: ' '.join(tup))
src_weekly_averages_mean_and_median

,z0_2m_c mean,z0_2m_c median,z0_3m_c mean,z0_3m_c median
time,,,,
2022-12-04,0.078178,0.068564,0.131898,0.122336
2022-12-11,0.025518,0.026357,0.045734,0.047682
2022-12-18,0.012628,0.003740,0.023706,0.005962
2022-12-25,0.024602,0.016075,0.041694,0.026104
2023-01-01,0.012506,0.004464,0.018285,0.005784
2023-01-08,0.013795,0.004391,0.024806,0.008523
2023-01-15,0.017884,0.004725,0.030577,0.006626
2023-01-22,0.005693,0.001233,0.008940,0.001253
2023-01-29,0.024167,0.005022,0.042206,0.006999


In [47]:
src[['z0_2m_c',	'z0_3m_c']].min(), src[['z0_2m_c',	'z0_3m_c']].max()

(variable
 z0_2m_c    0.000001
 z0_3m_c    0.000001
 dtype: float64,
 variable
 z0_2m_c    0.210880
 z0_3m_c    0.351702
 dtype: float64)

In [50]:
src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).mean().reset_index().min(),\
src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).median().reset_index().min()

(variable
 time       2022-12-02 00:00:00
 z0_2m_c               0.000096
 z0_3m_c               0.000043
 dtype: object,
 variable
 time       2022-12-02 00:00:00
 z0_2m_c               0.000031
 z0_3m_c               0.000014
 dtype: object)

In [49]:
src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).mean().reset_index().max(),\
src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).median().reset_index().max()

(variable
 time       2023-05-16 00:00:00
 z0_2m_c               0.123773
 z0_3m_c               0.229848
 dtype: object,
 variable
 time       2023-05-16 00:00:00
 z0_2m_c               0.123773
 z0_3m_c               0.229848
 dtype: object)

In [48]:
alt.Chart(
    src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).mean().reset_index()
).transform_fold(
    ['z0_2m_c',	'z0_3m_c']
).mark_line().encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Color("key:N")
).properties(width=1000) +\
alt.Chart(
    src[['z0_2m_c',	'z0_3m_c']].groupby(pd.Grouper(freq='1D')).median().reset_index()
).transform_fold(
    ['z0_2m_c',	'z0_3m_c']
).mark_line(strokeDash=[2,2]).encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Color("key:N")
).properties(width=1000) 

alt.LayerChart(...)

In [20]:
alt.Chart(
    src[['z0_2m_c',	'z0_3m_c']].reset_index()
).transform_fold(
    ['z0_2m_c',	'z0_3m_c']
).mark_circle().encode(
    alt.X('time:T'),
    alt.Y("value:Q").scale(type='log'),
    alt.Row("key:N")
).properties(width=1200)

alt.Chart(...)

# Solve for $z_0$ by fitting a line

In [31]:
y = src.iloc[0][
    ['spd_2m_c', 'spd_3m_c', 'spd_5m_c', 'spd_10m_c', 'spd_15m_c', 'spd_20m_c']
].values
x = np.array([np.log(2), np.log(3), np.log(5), np.log(10), np.log(15), np.log(20)])
regress_result = scipy.stats.linregress(x[:-1],y[:-1])
regress_result
xs = np.linspace(0.5, 3.5, 10)
ys = regress_result.slope*xs + regress_result.intercept

plt.scatter(x,y)
plt.plot(xs,ys)
u_star = regress_result.slope*von_karman
u_star
z0 = np.exp(-regress_result.intercept * von_karman / u_star)
z0

NameError: name 'scipy' is not defined

# Use Andreas et al. 2010 Method

In [ ]:
measurements = tidy_df_30Min.set_index("time").loc['2022-11-30': '2023-05-9']

$ u_* = \sqrt{\overline{u'w'}}$

In [ ]:
shear_velocity = (
    measurements.query("variable == 'u_w_rot__3m_c'")['value']
)**2

$L = \dfrac{\overline{\theta_v}}{k g}\dfrac{u_*^3}{\overline{w\theta_v}}$

where

$\overline{\Theta_v} = \text{average virtual temperature}$

$\overline{w \theta_v} = \text{flux of virtual temperature}$

In [ ]:
k = 0.4
g = 9.81 # m/s^2

obukhov_length = (
    measurements.query("variable == 'Tvirtual_3m_c'")['value'] / (k*g)
) * (
    shear_velocity**3 / measurements.query("variable == 'w_tc__3m_c'")['value']
)

In [ ]:
beta_g = 1.25
z_i = 600 #meters, assumed constant depth of the convective boundary layer - may not apply for our environment

w_star = shear_velocity*(
    - z_i / ( k * obukhov_length )
)**(1/3)

#unstable
s_r__unstable = (
    (
        measurements.query("variable == 'spd_3m_c'")['value']**2
    ) + (beta_g**2 * w_star**2)
)**0.5

s_r__stable = (
    measurements.query("variable == 'spd_3m_c'")['value'] + 0.5*(1 / np.cosh(
        measurements.query("variable == 'spd_3m_c'")['value']
    ))
)

$ C_{Dr} = \dfrac{u_*^2}{S_r^2} $

In [ ]:
C_Dr_unstable = shear_velocity**2 / s_r__unstable**2
C_Dr_stable = shear_velocity**2 / s_r__stable**2

In [ ]:
# obukhov_length
gamma = 16
z = 3

x_unstable = (1 - gamma*(z/obukhov_length))**0.25
phi_unstable = 2 * np.log((1 + x_unstable)/2) + np.log((1 + x_unstable**2)/2) - 2*np.arctan(x_unstable) + np.pi / 2

In [ ]:
a_m = 5
b_m = a_m/6.5
B_m = ((1 - b_m)/b_m)**(1/3)

In [ ]:
x_stable = (1 + z / obukhov_length)**(1/3)
phi_stable = (-3*a_m/b_m) * ( x - 1) + (a_m*B_m/(2*b_m))*(
    2*np.log(    (x_stable + B_m) / (z + B_m)   )
     - np.log(
            (x_stable**2 - x*B_m + B_m**2) / (1 - B_m + B_m**2)
     )
     + 2*np.sqrt(3) * (
         np.arctan(((2*x) - B_m) / (np.sqrt(3)*B_m)) - np.arctan( (2 - B_m) / (np.sqrt(3)*B_m))
        )
)

/home/elilouis/mambaforge/envs/arm/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
df = pd.DataFrame({
    'L' : obukhov_length,
    'C_Dr_stable': C_Dr_stable,
    'C_Dr_unstable': C_Dr_unstable,
    'phi_m_stable': phi_stable,
    'phi_m_unstable': phi_unstable,
})

In [ ]:
# if L < 0, unstable

df['C_Dr'] = df.apply(
    lambda row: row['C_Dr_unstable'] if row['L'] < 0 else row['C_Dr_stable'],
    axis=1
)
df['phi'] = df.apply(
    lambda row: row['phi_m_unstable'] if row['L'] < 0 else row['phi_m_stable'],
    axis=1
)

In [ ]:
df

,L,C_Dr_stable,C_Dr_unstable,phi_m_stable,phi_m_unstable,C_Dr,phi
time,,,,,,,
2022-11-30 00:00:00,3.294190e-13,2.441942e-11,NaN,NaN,NaN,2.441942e-11,NaN
2022-11-30 00:30:00,4.898748e-10,1.670684e-09,NaN,NaN,NaN,1.670684e-09,NaN
2022-11-30 01:00:00,7.328991e-15,1.550992e-12,NaN,NaN,NaN,1.550992e-12,NaN
2022-11-30 01:30:00,3.352121e-20,3.295582e-16,NaN,NaN,NaN,3.295582e-16,NaN
2022-11-30 02:00:00,2.486688e-13,3.070947e-11,NaN,NaN,NaN,3.070947e-11,NaN
...,...,...,...,...,...,...,...
2023-05-09 21:30:00,-1.643095e-16,3.343604e-13,1.028222e-13,NaN,36.565915,1.028222e-13,36.565915
2023-05-09 22:00:00,-3.086058e-18,2.097586e-14,6.968017e-15,NaN,40.540664,6.968017e-15,40.540664
2023-05-09 22:30:00,-1.686784e-19,4.757312e-15,1.145390e-15,NaN,43.447287,1.145390e-15,43.447287


$ z_0 = r \exp{\Big(
    - ( k*C_{dr}^{(-1/2)} + \psi_m(r/L))    
\Big)} $

In [ ]:
df['z0'] = z * np.exp(
    - (k * df['C_Dr']**(-0.5) + df['phi'])
)

In [ ]:
df

NameError: name 'df' is not defined